# Purpose
- abstract loading configs from user
- Improve testability by using composition:
  - StepFactory "has a" StepConfigFactory
  - -> For testing, we can easily compose a StepFactory with a mock version of the StepConfigFactory, rather than having to monkey-patch the get_configs method, etc.

# Decisions:
## Instantiate factory with step name and environment, or pass these variables in create_configs method?

In [ ]:
class StepConfigFactoryInterface:
    @abstractmethod
    def get_configs() -> StepConfigFacade:
        ...

class FWPConfigFactory(StepConfigFactoryInterface):
    def __init__(self, env: Environment, step_name: str) -> None:
        self.env = env
        self.step_name = step_name

    def get_configs(self) -> FrameworkProcessingConfigFacade:
        return fw_proc_configs

In [ ]:
class StepConfigFactoryInterface:
    @abstractmethod
    def get_configs(self, env: Environment, step_name: str) -> StepConfigFacade:
        ...

class FWPConfigFactory(StepConfigFactoryInterface):
    def __init__(self) -> None:
        pass

    def get_configs(self, env: Environment, step_name: str) -> FrameworkProcessingConfigFacade:
        return fw_proc_configs

- ~~I can't see any difference from a design patterns perspective. Either one should work about the same. The second option would allow using the same factory for different steps, but at least with the current design I can't really see any advantage in that.~~
- ~~Decision: Instantiate factory about step name and environment, as there seems to slightly simplify the code.~~

**Decision: Pass step name and environment as arguments and create_configs method, so that config factory does not have to know which step it is owned by.**

- Step **config factory can be owned by the pipeline façade.** 
- Advantages:
  - only need to instantiate a single config factory for the whole pipeline, which simplifies code.
  - Makes it easier to also load shared_config through factory, which simplifies code. (Reason it's easier is that we don't have to worry about how to avoid loading config from file just once.)
- Implications:
  - Façade can load shared can during instantiation
  - Didn't find an ideal way for loading step config. Either load during instantiation and store in a dictionary, or as needed for a given step. But either way, the challenge is that the step has to reach back out to the façade to get the config, because the façade does not know that name (at least in the current design), so it cannot pass config in method call to step factory. Current solution: 
    - Façade reaches out to step factory to get step name. ( -> step_name needs to be part of public interface)
    - Façade loads config for that step name.
    - Façade creates step from step factory, passing step config loaded above (as well as shared config) as argument in the  method call.